# Daily Power Generation in India

### India is the world's third-largest producer and third largest consumer of electricity.

India's electricity sector is dominated by fossil fuels, in particular coal, which during the 2018-19 fiscal year produced about three-quarters of the country's electricity. The government is making efforts to increase investment in renewable energy. The government's National Electricity Plan of 2018 states that the country does not need more non-renewable power plants in the utility sector until 2027, with the commissioning of 50,025 MW coal-based power plants under construction and addition of 275,000 MW total renewable power capacity after the retirement of nearly 48,000 MW old coal-fired plants.

India has recorded rapid growth in electricity generation since 1985, increasing from 179 TW-hr in 1985 to 1,057 TW-hr in 2012. The majority of the increase came from coal-fired plants and non-conventional renewable energy sources (RES), with the contribution from natural gas, oil, and hydro plants decreasing in 2012-2017. The gross utility electricity generation (excluding imports from Bhutan) was 1,384 billion kWh in 2019-20, representing 1.0 % annual growth compared to 2018-2019. The contribution from renewable energy sources was nearly 20% of the total. In the year 2019-20, all the incremental electricity generation is contributed by renewable energy sources as the power generation from fossil fuels decreased.

### Please visit this [link](https://npp.gov.in/dashBoard/cp-map-dashboard) for more interesting Dashboards from [https://npp.gov.in/](https://npp.gov.in/)

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas_profiling
import os
import calendar
pio.templates.default = "plotly_dark"
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/daily-power-generation-in-india-20172020/india_states.geojson
/kaggle/input/daily-power-generation-in-india-20172020/State_Region_corrected.csv
/kaggle/input/daily-power-generation-in-india-20172020/file_02.csv
/kaggle/input/daily-power-generation-in-india-20172020/region_cordinates.csv


In [2]:
data1 = pd.read_csv('../input/daily-power-generation-in-india-20172020/file_02.csv')
data1.head()

,index,Date,Region,Thermal Generation Actual (in MU),Thermal Generation Estimated (in MU),Nuclear Generation Actual (in MU),Nuclear Generation Estimated (in MU),Hydro Generation Actual (in MU),Hydro Generation Estimated (in MU)
0,0,2017-09-01,Northern,624.23,484.21,30.36,35.57,273.27,320.81
1,1,2017-09-01,Western,"1,106.89","1,024.33",25.17,3.81,72.00,21.53
2,2,2017-09-01,Southern,576.66,578.55,62.73,49.80,111.57,64.78
3,3,2017-09-01,Eastern,441.02,429.39,NaN,NaN,85.94,69.36
4,4,2017-09-01,NorthEastern,29.11,15.91,NaN,NaN,24.64,21.21


### Data Preprocessing

In [3]:
data1['Date'] = pd.to_datetime(data1['Date'])

In [4]:
data1['Thermal Generation Estimated (in MU)'] = data1['Thermal Generation Estimated (in MU)'].str.replace(',','').astype('float')
data1['Thermal Generation Estimated (in MU)'].values

array([ 484.21, 1024.33,  578.55, ...,  415.53,  547.03,   32.81])

In [5]:
data1['Thermal Generation Actual (in MU)'] = data1['Thermal Generation Actual (in MU)'].str.replace(',','').astype('float')
data1['Thermal Generation Actual (in MU)'].values

array([ 624.23, 1106.89,  576.66, ...,  494.66,  482.86,   34.42])

In [6]:
def time_series_overall(df, groupby, dict_features, filter=None):
    temp = df.groupby(groupby).agg(dict_features)
    fig = go.Figure()
    for f,c in zip(dict_features, px.colors.qualitative.D3):
        fig.add_traces(go.Scatter(y=temp[f].values,
                              x=temp.index,
                              name=f,
                              marker=dict(color=c)
                             ))
    fig.update_traces(marker_line_color='rgb(255,255,255)',
                      marker_line_width=2.5, opacity=0.7)
    fig.update_layout(
                    width=1000,
                    xaxis=dict(title="Date", showgrid=False),
                    yaxis=dict(title="MU", showgrid=False),
                    legend=dict(
                                x=0,
                                y=1.2))
                                
    fig.show()

In [7]:
dict_features = {
    "Thermal Generation Estimated (in MU)": "sum",
    "Thermal Generation Actual (in MU)": "sum",
   
}
time_series_overall(data1, groupby="Date", dict_features=dict_features)

dict_features = {
    "Nuclear Generation Estimated (in MU)": "sum",
    "Nuclear Generation Actual (in MU)": "sum",
}
time_series_overall(data1, groupby="Date", dict_features=dict_features)

dict_features = {
     "Hydro Generation Estimated (in MU)": "sum",
    "Hydro Generation Actual (in MU)": "sum"
}
time_series_overall(data1, groupby="Date", dict_features=dict_features)

### Report - 

- These graphs clearly show that the Actual Power Generated is much higher than the Estimated one!

## State wise Visualisation of National Share

In [8]:
state_df = pd.read_csv('/kaggle/input/daily-power-generation-in-india-20172020/State_Region_corrected.csv')
state_df.head()

,State / Union territory (UT),Area (km2),Region,National Share (%)
0,Rajasthan,342239,Northern,10.55
1,Madhya Pradesh,308350,Central,9.37
2,Maharashtra,307713,Western,9.36
3,Uttar Pradesh,240928,Northern,7.33
4,Gujarat,196024,Western,5.96


## Region wise representation of Nation wise Power Distribution In India

In [9]:
state = state_df.groupby('Region')['National Share (%)'].sum().sort_values(ascending = False)
state.index
fig = px.pie(state,values = state.values, names=state.index,
            title='Distribution of Power')
fig.show()

## State wise representation of Nation wise Power Distribution

In [10]:
fig = px.bar(state_df.nlargest(15, "National Share (%)"),
       x = 'National Share (%)',
       y = 'State / Union territory (UT)',
       text="National Share (%)",
      color ='State / Union territory (UT)')
fig.show()

**Hey Kagglers, if you found this Visualisation Interesting and Insightful, do upvote and let me know your thoughts on it!**